In [2]:
%%writefile C:\Users\Cflhxb\AppData\Local\anaconda3\Lib\GraphHelpers.py

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict
    
def SetGraphicsPatterns(ColumnIndex,F1Lines,F1Alphas,F1Markers,F2Colors,InvertLevels = False):
    """Sets up series and data frames linking each treatment to graph colors and linestyles

    Args:
        ColumnIndex: A two level multiindex from the data frame holding data to be graphed.  
        F1Lines: Series of line properties (in string format) used to demark F1
        F1Alphas: Series of transperiencies (0-1) use to demark range fills
        F2Colors: Series of color properties (in string format) used to demark F2
        GPs: the Graphics Patterns classes that store the set properties
        InvertLevels: If true will apply colours to F1 and lines/shapes to F2
    Returns:
        Value of net solar radiation
    """
    class GPs:
        F1Levels = []
        F2Levels = []
        Treatments = []
        Styles = []
        StyleCycle = []
        StyleFrame = pd.MultiIndex
        TreatCombs = []
        ColorCycle = []
        LineCycle = []
    
    if InvertLevels == False:
        GPs.F1Levels = list(ColumnIndex.get_level_values(0).drop_duplicates())
        GPs.F2Levels = list(ColumnIndex.get_level_values(1).drop_duplicates())
    else:
        GPs.F1Levels = list(ColumnIndex.get_level_values(1).drop_duplicates())
        GPs.F2Levels = list(ColumnIndex.get_level_values(0).drop_duplicates())
    print(ColumnIndex.names[0])
    print(GPs.F1Levels)
    print(ColumnIndex.names[1])
    print(GPs.F2Levels)
    GPs.Treatments = pd.MultiIndex.from_product([GPs.F1Levels,GPs.F2Levels])
    print('Treatments')
    print(GPs.Treatments)
    F1styles = [(F1Lines[x],F1Alphas[x],F1Markers[x]) for x in range(len(F1Lines))]
    if InvertLevels == False:
        GPs.Styles = [(x[0],y,x[1],x[2]) for x in F1styles for y in F2Colors]
    else:
        GPs.Styles = [(x[0],y,x[1],x[2]) for y in F2Colors for x in F1styles]
    print('Styles')
    print(GPs.Styles)
    def MFC(ind):
        if InvertLevels == False:
            if ind[0] == GPs.F1Levels[0]:
                return GPs.StyleFrame.loc[ind,'Col']
            else:
                return 'white'
        else:
            if ind[1] == GPs.F2Levels[0]:
                return GPs.StyleFrame.loc[ind,'Col']
            else:
                return 'white'
    GPs.StyleFrame = pd.DataFrame(index=GPs.Treatments,columns=['Lin','Col','Alp','Mar'],data = GPs.Styles)
    GPs.StyleFrame.loc[:,'MFC'] = [MFC(x) for x in GPs.StyleFrame.index]
    print('StyleFrame')
    print(GPs.StyleFrame)

    return GPs

def GetIndexedColorCycle(ColumnIndex,GPs):
    """Returns a series of colours corresponding to the ColumnIndex and Graphics patterns passed
    
    Args:
        ColumnIndex: A two level multiindex from the data frame holding data to be graphed.  
        GPs: the Graphics Patterns classes that store the set properties
    """
    
    F2Levels = ColumnIndex.get_level_values(1)
    colorCycle = []
    for F2 in F2Levels:
        colorCycle.append(GPs.StyleFrame.loc[(slice(None),F2),'Col'][0])
    return colorCycle    

def GetIndexedLineCycle(ColumnIndex,GPs):
    """Returns a series of line styles corresponding to the ColumnIndex and Graphics patterns passed
    
    Args:
        ColumnIndex: A two level multiindex from the data frame holding data to be graphed.  
        GPs: the Graphics Patterns classes that store the set properties
    """
    
    F1Levels = ColumnIndex.get_level_values(0)
    lineCycle = []
    for F1 in F1Levels:
        lineCycle.append(GPs.StyleFrame.loc[F1,'Lin'][0])
    return lineCycle    

def GetIndexedMarkerCycle(ColumnIndex,GPs):
    """Returns a series of marker styles corresponding to the ColumnIndex and Graphics patterns passed
    
    Args:
        ColumnIndex: A two level multiindex from the data frame holding data to be graphed.  
        GPs: the Graphics Patterns classes that store the set properties
    """
    
    F1Levels = ColumnIndex.get_level_values(0)
    markerCycle = []
    for F1 in F1Levels:
        markerCycle.append(GPs.StyleFrame.loc[F1,'Mar'][0])
    return markerCycle

def AddIndexedPlot(Data,ax,GPs,LW):
    """Add a plot to ax with line color and style corresponding to those 
    specified in GraphicsPatterns class.  Will plot multiple lines 
    (if dataframe has more than one column) with the same style properties

    Args:
        Data: Pandas dataframe with a two level column multi index,
        ax: the axis object to plot onto
        GPs: The graphics properties class with stiles to apply
        LW: (int) line width for plot
    """
    F1 = Data.columns.get_level_values(0).drop_duplicates().values
    F2 = Data.columns.get_level_values(1).drop_duplicates().values
    for L1 in F1:
        for L2 in F2:
            ax.plot(Data.loc[:,(L1,L2)],lw=LW,
                             linestyle = GPs.StyleFrame.loc[(L1,L2),'Lin'],
                             color = GPs.StyleFrame.loc[(L1,L2),'Col'],
                   label = L1 + " " + L2)
def AddLegend(LegLoc, labelsize, Title, NCol = 1,MScale=1):
    """Add a tidy (duplicates removed) legend to current plot

    Args:
        LegLoc: (1,2,3,4) location of the legend,
        lablesize: size of font
        Title: title of legend
    """
    labels, handles = DedupLegendElements()
    ledg = plt.legend(labels, handles,loc=LegLoc,numpoints=1,fontsize = labelsize,labelspacing = 0.05, 
                          title=Title,ncol=NCol,markerscale=MScale)
    ledg.get_title().set_fontsize(labelsize + 2)
            
def AddIndexedPlotSingleLevel(Data,ax,GPs,LW):
    """Add a plot to ax with line color corresponding to those 
    specified in GraphicsPatterns class.  Will plot multiple lines 
    (if dataframe has more than one column) with the same style properties

    Args:
        Data: Pandas dataframe with a two level column multi index,
        ax: the axis object to plot onto
        GPs: The graphics properties class with stiles to apply
        LW: (int) line width for plot
    """
    F1 = Data.columns.get_level_values(0).drop_duplicates().values
    for L1 in F1:
        ax.plot(Data.loc[:,L1],lw=LW,
                         linestyle = GPs.StyleFrame.loc[(slice(None),L1),'Lin'][0],
                         color = GPs.StyleFrame.loc[(slice(None),L1),'Col'][0],
               label = L1)

def AddIndexedXY(DataX,DataY,ax,GPs,MS):
    """Add a plot to ax with marker color and style corresponding to those 
    specified in GraphicsPatterns class.  Will plot multiple sets of points 
    (if dataframe has more than one column) with the same style properties.  
    DataX and DataY frames must have the same column index levels

    Args:
        DataX: Pandas dataframe with a two level column multi index containing data to be the x variable,
        DataY: Pandas dataframe with a two level column multi index containing data to be the y variable,
        ax: the axis object to plot onto
        GPs: The graphics properties class with stiles to apply
        MS: (int) Marker size
    """
    try:
        F1 = DataX.columns.get_level_values(0).drop_duplicates().values
    except:
        F1 = DataY.columns.get_level_values(0).drop_duplicates().values
    try:
        F2 = DataX.columns.get_level_values(1).drop_duplicates().values
    except:
        F2 = DataY.columns.get_level_values(1).drop_duplicates().values
    for L1 in F1:
        for L2 in F2:
            try: #Getting x from index of passed DataX
                X = DataX.loc[:,(L1,L2)]
            except:
                try: #Using passed DataX assuming the same x data used for all treatments
                    X =  DataX
                except:
                    print ('Error: plot failed for ' + L1 + " " + L2)
                    pass
                
            try: #Getting y from index of passed DataY
                Y = DataY.loc[:,(L1,L2)]
            except:
                try: #Using passed DataY assuming same y data used for all treatments
                    Y = DataY
                except:
                    print ('Error: plot failed for ' + L1 + " " + L2)
                    pass
            else: 
                ax.plot(X,Y, linestyle = 'None', ms = MS,
                        marker = GPs.StyleFrame.loc[(L1,L2),'Mar'],
                        mec = GPs.StyleFrame.loc[(L1,L2),'Col'],
                        mfc = GPs.StyleFrame.loc[(L1,L2),'MFC'],
                        label = L1 + " " + L2)
            
            
def AddIndexedPlotWithSEM(Data,ax,GPs,LW):
    """Add a plot to ax with line (median) and range (SEM) colors and styles 
    corresponding to those specified in GraphicsPatterns class.  
    Needs to have multiple columns in Data to calculate ranges

    Args:
        Data: Pandas dataframe with a two level column multi index and two or more columns
        ax: the axis object to plot onto
        GPs: The graphics properties class with stiles to apply
        LW: (int) line width for plot
    """
    F1 = Data.columns.get_level_values(0).drop_duplicates().values
    F2 = Data.columns.get_level_values(1).drop_duplicates().values
    for L1 in F1:
        for L2 in F2:
            TD = Data.loc[:,(L1,L2)]
            Med = TD.median(axis=1)
            SEM = TD.std(axis=1)/np.sqrt(TD.columns.size)
            Upp = Med + SEM
            Low = Med - SEM
            ax.fill_between(TD.index,Upp,Low,lw=0,
                           color = GPs.StyleFrame.loc[(L1,L2),'Col'],
                           alpha = GPs.StyleFrame.loc[(L1,L2),'Alp'])
            ax.plot(TD.index,Med,lw=LW,
                             linestyle = GPs.StyleFrame.loc[(L1,L2),'Lin'],
                             color = GPs.StyleFrame.loc[(L1,L2),'Col'],
                   label = L1 + " " + L2)

def SortFrame(Frame):
    """Function to sort specified dataframe on both indexes"""
    Frame.sort_index(axis=0,inplace=True)
    Frame.sort_index(axis=1,inplace=True)
    
def DedupLegendElements():
    """Removes duplicate elements form legend and returns a series called legends
    which is the deduplicated list of names and a series called handles which is 
    the corresponding list of legend keys"""
    
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    labels = by_label.values()
    handles = by_label.keys()
    return labels, handles

def SetAxProps(title='',titsize = 20,ylab = '',xlab='', labsize=18, ticsize = 16, xlim=(None,None),ylim=(None,None)):
    plt.title(title,fontsize=titsize)
    plt.ylabel(ylab,fontsize=labsize)
    plt.xlabel(xlab,fontsize=labsize)
    plt.tick_params(labelsize = ticsize)
    plt.xlim(xlim)
    plt.ylim(ylim)


Overwriting C:\Users\Cflhxb\AppData\Local\anaconda3\Lib\GraphHelpers.py
